### Inverted Index Construction and Preprocessing

In [ ]:
import nltk
import textract
from nltk.corpus import *
from nltk.stem.porter import *
import os
import pickle
from nltk.tokenize import sent_tokenize, word_tokenize
from pathlib import Path


Before constructing the inverting index we need to install essential datasets for stopword removal and normalization done during the preprocessing step. We have used the available nltk datasets for the same purpose.

In [ ]:
# ! Uncomment in first run 
# nltk.download('punkt')
# nltk.download('words')
# nltk.download('stopwords')

### Normalization
Porter stemming algorithm is used for the purpose of normalization of tokenized words. It reduces a given term to its root form, thus the query is robust to more typical morphological and inflexional endings. We have the used the PorterStemmer by nltk library for the same purpose. 

In [ ]:
# stemming porter object
stemmer = PorterStemmer()
root = Path(".")


Before construction of inverted index, we have converted all the pdf files given to word documents as directly reading from pdfs lead to a higher loss as compared to word documents and the pdfs might contain watermarks or unselectable text which can hamper our text extraction purpose. We have used **textract** and **sent_tokenize** by nltk library to extract the raw text from the documents and tokenize every sentence to preserve the context. 

In [ ]:
# list of all doc names
files = list()
for dir in [r"\Auto", r"\Property"]:
    cur_dir = r".\Docs" + dir
    for file in os.listdir(cur_dir):
        cur_path = r".\Docs" + dir + "\\" + file
        files.append(cur_path)
files.sort()

In [ ]:
docs = list()
for x in range(len(files)):
    for i in sent_tokenize(textract.process(files[x]).decode("utf8")): 
        docs.append((i, x))

### Inverted Index Construction
The structure of our inverted index is a list of tuples of form *(doc index, file index, frequency)* where doc index is the index of the newly transformed documents, file index is the index of file the transformed document belongs to, and the frequency is the number of times the normalized term or the **key** appears in the transformed document. The size of our transformed documents is dynamic in nature, the minimum threshold we have kept is 500 characters. We keep on adding sentences to a document until we get atleast 500 characters. Benefit of adding sentences instead of words is that context of each transformed document is preserved, it starts with a sentence and there is no abrupt ending of the context when the user queries for any document.

For a given sentence, we tokenize it to words using **nltk.word_tokenize** and convert those words to lower case. if the word is a stopword, we skip it, otherwise we normalize the word using porter stemming and construct the inverted index with **key** as the normalized term. Our inverted_index is a dictionary whose **key** is the normalized term and **value** is the list of tuples as mentioned above.

After the construction of the inverted index table, we sort each list by the third element of the tuple, that is frequency of the **key** so that we get relevant results whenever user queries for anything.

In [ ]:
# set of Stop words
stop_words = set(stopwords.words('english'))

"""
{
   key : string (normalized)
   value: list of ("doc index", file_index, frequency) 
}
"""
inverted_idx = dict()

# list of string modified document
documents = list()

count_id = 0

def process(doc_index):
    """
    Reads file, tokenize it, normalizes it and builds the inverted index
    """

    result = doc_index + 1
    global count_id
    text = docs[doc_index][0]
    file_index = docs[doc_index][1]
    while doc_index + 1 < len(docs):
        doc_index += 1
        if docs[doc_index][1] == docs[doc_index - 1][1] and len(text + docs[doc_index][0]) <= 500:
            text += docs[doc_index][0]
        else:
            result = doc_index
            break

    tokens = nltk.tokenize.word_tokenize(str(text))

    new_token = list()
    for i in tokens:
        new_token.append(i.lower())
    tokens = new_token

    curr_str = ""
    normalised_word_freq = dict()
    for j in range(len(tokens)):
        curr_str += tokens[j] + " "

        normal = stemmer.stem(tokens[j].lower())
        if normalised_word_freq.get(normal) != None:
            normalised_word_freq[normal] += 1
        else:
            normalised_word_freq[normal] = 1 

    documents.append((curr_str, files[file_index]))
    
    visited = set()
    for j in range(len(tokens)):
        normalised_word = stemmer.stem(tokens[j].lower())
        if tokens[j].lower() not in stop_words and normalised_word not in visited:
            visited.add(normalised_word)

            if inverted_idx.get(normalised_word) != None:
                inverted_idx[normalised_word].append((count_id, file_index, normalised_word_freq[normalised_word]))
            else:
                inverted_idx[normalised_word] = [(count_id, file_index, normalised_word_freq[normalised_word])]
    count_id += 1

    return result

i = 0
while i < len(docs):
    i = process(i)

for x in inverted_idx:
    inverted_idx[x] = sorted(inverted_idx[x], key=lambda y: -y[2])


After the entire preprocessing stage inverted index table, transformed document list and the list of original files is dumped to respective binary files so that whenever user wants to query, our search engine can just open those files and use those data structures for querying directly instead of preprocessing again and again.

In [ ]:
my_path = root / "Pickled_files" / "Inverted_index"
dbfile = open(my_path, 'wb')
pickle.dump(inverted_idx, dbfile) 
dbfile.close()

my_path = root / "Pickled_files" / "Documents"
dbfile = open(my_path, 'wb')
pickle.dump(documents, dbfile) 
dbfile.close()

my_path = root / "Pickled_files" / "Files"
dbfile = open(my_path, 'wb')
pickle.dump(files, dbfile) 
dbfile.close()